In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('c:\\Users\\Ken\\Documents\\GitHub\\coursera_rag\\requirements_from_acts.csv')
df.fillna('blank', inplace=True)  # remove any NaN values as it blows up serialization
data = df.to_dict('records')
len(data)

227

In [27]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [28]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [29]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [30]:
# Create collection to store records
qdrant.recreate_collection(
    collection_name="legal_reqts",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [31]:
# vectorize
qdrant.upload_points(
    collection_name="legal_reqts",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Essential Requirement"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the records
    ]
)

In [33]:
# structure the database search
user_prompt = "Identify all requirements for risk analysis"

In [34]:
# Search source

hits = qdrant.search(
    collection_name="legal_reqts",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

# write the list to a file
file_path = 'C:\\Users\\Ken\\Downloads\\source.txt'

with open(file_path, 'w') as file:
  for hit in hits:
    # print(hit.payload, "score:", hit.score)
    file.write(str(hit.payload)+','+str(hit.score)+'\n')

print(f"The 'hits' list has been written to {file_path}.")

The 'hits' list has been written to C:\Users\Ken\Downloads\source.txt.


In [35]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
# Start the llamafile server

import subprocess

# Command to start the llamafile server
server_process = subprocess.Popen(['C:\\Users\\Ken\\Documents\\GitHub\\coursera_rag\\.venv\\llava-v1.5-7b-q4.llamafile.exe'])

In [36]:
# Connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a chatbot, skilled in explaining regulatory requirements for machinery."},
        {"role": "user", "content": "Identify requirements for risk analysis as part of CE marking."},
        
        {"role": "assistant", "content": str(search_results)}
    ]
)

output_message = completion.choices[0].message
import csv

# write output to CSV
output = [
    {"role": "system", "content": "You are a chatbot, skilled in explaining regulatory requirements for machinery."},
    {"role": "user", "content": "Identify requirements for risk analysis as part of CE marking."},
    {"role": output_message.role, "content": output_message.content}
]
# Write data to a CSV file
with open("C:\\Users\\Ken\\Downloads\\output.csv", "w", newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["role", "content"])
    writer.writeheader()
    writer.writerows(output)

print("Output written to file")

Output written to file


In [37]:
# Terminate the llamafile server

import signal

server_process.send_signal(signal.SIGTERM)